# Creating Project

Firstly, you need to create a project on Google Cloud Platform to work on

![Create project](./images/create_project.png)

![Name project](./images/name_project.png)

# Create Workbench

Go to Vertex AI page then click `Create Workbench` using the default settings. After creating, copy the 2 notebooks into the workbench.

![Workbench](./images/workbench.png)

# Setup

In [3]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'tymestack-439300'

In [5]:
REGION = 'us-central1'
EXPERIMENT = '01'
SERIES = '02'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'housing'
BQ_TABLE = 'housing'

data_url = "http://lib.stat.cmu.edu/datasets/boston"

In [33]:
from google.cloud import bigquery
from google.cloud import storage

In [34]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

In [35]:
BUCKET = PROJECT_ID

# Data processing

In [46]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
data = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

In [47]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.00632,18.00,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3
1,396.90000,4.98,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02731,0.00,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8
3,396.90000,9.14,21.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.02729,0.00,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8


In [48]:
data.shape

(1012, 11)

In [49]:
data = data.dropna()

In [50]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000


In [51]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
print(data.describe())

               0           1           2           3           4           5   \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     0.040544    0.113636    0.391378    0.069170    0.349167    0.521869   
std      0.096679    0.233225    0.251479    0.253994    0.238431    0.134627   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000851    0.000000    0.173387    0.000000    0.131687    0.445392   
50%      0.002812    0.000000    0.338343    0.000000    0.314815    0.507281   
75%      0.041258    0.125000    0.646628    0.000000    0.491770    0.586798   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

               6           7           8           9           10  
count  506.000000  506.000000  506.000000  506.000000  506.000000  
mean     0.676364    0.242381    0.371713    0.422208    0.622929  
std      0.289896    0.191482    0.378576    0.321636    0.230313 

# Store the Source Data in GCS Storage Bucket

In [52]:
import os

# Define the local directory and file path
local_dir = "./tmp"
csv_file = os.path.join(local_dir, "boston_housing.csv")

# Create the directory if it doesn't exist
if not os.path.exists(local_dir):
    os.makedirs(local_dir)

# Save the DataFrame as a CSV
data.to_csv(csv_file, index=False)
print(f"File saved locally as {csv_file}")

File saved locally as ./tmp/boston_housing.csv


In [53]:
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

In [61]:
# Initialize the GCS client
bucketDef = gcs.bucket(BUCKET)

# Check if the file already exists in the GCS bucket
if storage.Blob(bucket=bucketDef, name=file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    # Upload the local CSV file to the GCS bucket
    blob = bucketDef.blob(file)
    blob.upload_from_filename(csv_file)  # Uploads the local CSV file
    
    print(f'Uploaded the CSV file to: gs://{bucketDef.name}/{file}')

The file has already been created at: gs://tymestack-439300/01/01/data/housing.csv


In [62]:
list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}'))

[<Blob: tymestack-439300, 01/01/data/fraud.csv, 1729473291541936>,
 <Blob: tymestack-439300, 01/01/data/housing.csv, 1729491139511909>,
 <Blob: tymestack-439300, 01/01/training/01_trainer/train.py, 1729498657541273>,
 <Blob: tymestack-439300, 01/01/training/Dockerfile, 1729498659629682>,
 <Blob: tymestack-439300, 01/01/training/requirements.txt, 1729498659421529>]

# Create BigQuery Dataset

In [56]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud


In [57]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

In [58]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud
housing


# Create BigQuery Table

In [59]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Creating Table ...
Finished creating table: tymestack-439300.housing.housing


In [60]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


,double_field_0,double_field_1,double_field_2,double_field_3,double_field_4,double_field_5,double_field_6,double_field_7,double_field_8,double_field_9,double_field_10
0,0.000210,0.00,0.052419,0.0,0.273663,0.570799,0.584964,0.467159,0.000000,0.448473,0.351064
1,0.000633,0.00,0.420455,0.0,0.386831,0.580954,0.681771,0.122671,0.000000,0.164122,0.893617
2,0.000438,0.00,0.420455,0.0,0.386831,0.490324,0.760041,0.105293,0.000000,0.164122,0.893617
3,0.000612,0.00,0.420455,0.0,0.386831,0.654340,0.907312,0.094381,0.000000,0.164122,0.893617
4,0.001161,0.00,0.420455,0.0,0.386831,0.619467,0.889804,0.114514,0.000000,0.164122,0.893617
...,...,...,...,...,...,...,...,...,...,...,...
502,0.000531,0.21,0.189883,0.0,0.111111,0.565242,0.187436,0.516973,0.130435,0.106870,0.446809
503,0.000489,0.21,0.189883,0.0,0.111111,0.466948,0.190525,0.516973,0.130435,0.106870,0.446809
504,0.000401,0.28,0.534457,0.0,0.162551,0.552021,0.522142,0.230638,0.130435,0.158397,0.595745
505,0.000252,0.28,0.534457,0.0,0.162551,0.507760,0.267765,0.230638,0.130435,0.158397,0.595745
